In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('DATATHON_EVENT_DATASET.csv')

In [ ]:
data.head()

In [ ]:
# data = data.drop("Expected_Fraud", axis=1)

In [ ]:
data.dtypes

In [ ]:
data['TransactionAmount'] = - data['Final_Origin_Balance'] + data['Initial_Origin_Balance']

In [ ]:
# data = data.drop(['Initial_Origin_Balance', 'Initial_Destination_Balance'], axis=1)

In [ ]:
data.columns

In [ ]:
# # Function to hash Origin_ID
# def hash_origin_id(origin_id):
#     return hashlib.sha256(origin_id.encode()).hexdigest()

# # Apply the function to the Origin_ID column
# data['Hashed_Origin_ID'] = data['Origin_ID'].apply(hash_origin_id).rank(method="dense").astype(int)
# data['Hashed_Destination_ID'] = data['Destination_ID'].apply(hash_origin_id).rank(method="dense").astype(int)

encodables = ['Origin_ID', 'Destination_ID', 'Fraud']
encoders = {}
for column in encodables:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    encoders[column] = le

In [ ]:
data.head()

In [ ]:
# data = data.drop(['Origin_ID', 'Destination_ID'], axis=1)

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
data['Destination_ID'].value_counts()

In [ ]:
data['Origin_ID'].value_counts()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
(data['Initial_Origin_Balance'] - data['Final_Origin_Balance']) == (data['Final_Destination_Balance'] - data['Initial_Destination_Balance'])


In [ ]:
# data = data.drop(['Origin_ID', 'Destination_ID', 'Initial_Origin_Balance', 'Initial_Destination_Balance'], axis=1)

In [ ]:
data.head()

In [ ]:
data['Transaction_Type'].value_counts()

In [ ]:
transaction_types = data['Transaction_Type'].unique()
print(transaction_types)

In [ ]:
# No null values are found except for the Expected_fraud column
null_count = data.isnull().sum()
print(null_count)  # Returns the count of nulls

In [ ]:
# Create a mapping dictionary for the similar labels
transaction_type_mapping = {
    'CASH_OUUUT': 'CASH_OUT',
    'Trans_fer': 'TRANSFER',
    'TRANSFER': 'TRANSFER',  # Keeping it the same for consistency
    'CASH_OUT': 'CASH_OUT',  # Keeping it the same
    'PAYMENT': 'PAYMENT',  # Keeping it the same
    'CASH_IN': 'CASH_IN',  # Keeping it the same
}

In [ ]:
# Apply the mapping to replace inconsistent labels with correct ones
data['Transaction_Type'] = data['Transaction_Type'].replace(transaction_type_mapping)

# Verify the changes
print(data['Transaction_Type'].value_counts())
# #Transaction amount is negative if outgoing and positive for incoming

In [ ]:
# # Assuming your DataFrame is called 'data'

# # Step 1: Create a new column 'Transaction_Sign' based on the sign of the 'Amount'
# data['Transaction_Amount_sign'] = data['TransactionAmount'].apply(lambda x: 0 if x < 0 else 1)

# # Step 2: Group the data by 'Transaction_Type' and 'Transaction_Sign' a
# grouped_data = data.groupby(['Transaction_Type', 'Transaction_Amount_sign']).size().unstack(fill_value=0)

# # Display the grouped data
# #print(grouped_data)
# # Assuming the result is stored in 'grouped_data'
# reversed_axes = grouped_data.transpose()

# # Display the reversed axes DataFrame
# print(reversed_axes)

In [ ]:
data.head()

In [ ]:
data = pd.get_dummies(data, columns=['Transaction_Type'])
data.head()

In [ ]:
data['source_amount'] = (data['Initial_Origin_Balance'] - data['Final_Origin_Balance'])
data['destination_amount'] = (data['Final_Destination_Balance'] - data['Initial_Destination_Balance'])

In [ ]:

money_columns = ['Amount', 'source_amount', 'destination_amount']

In [ ]:
data[(data['source_amount'].abs() == data['destination_amount'].abs()) & (data['source_amount'] != data['Amount'].abs())][money_columns].head()

In [ ]:
swappables = (data['source_amount'] <= 0) & (data['destination_amount'] <= 0)
data.loc[swappables, 'source_amount'], data.loc[swappables, 'destination_amount'] = data.loc[swappables, 'destination_amount'], data.loc[swappables, 'source_amount']

In [ ]:
data['time_of_day'] = data['Time']%24
data.groupby('Fraud')['time_of_day'].value_counts()


In [ ]:
data.groupby('Fraud')['Time'].value_counts()


In [ ]:
columns_to_drop = [
    'Origin_ID', 'Destination_ID', 'Initial_Origin_Balance', 
    'Initial_Destination_Balance', 'Expected_Fraud', 
    'source_amount', 'destination_amount', 'TransactionAmount', 'time_of_day', 'Time'
]

# Drop columns if they exist
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])

data.head()

In [ ]:
data.head()

In [ ]:
data.to_csv('cleaned_data.csv', index=False)